This notebook reproduces the **geometric and positional analyses** of droplets described in the manuscript.  

## Analysis Workflow

- **Segmentation**
  - Droplets segmented by intensity thresholding followed by contour detection.
  - Largest connected component selected as the droplet region of interest.

- **Size Measurement**
  - Droplet size determined from cross-sectional area at the manually chosen central plane.
  - Radius analyzed from major and minor axes length

- **Position Measurement**
  - Two manual reference points annotated on the radial axis through the droplet centroid:
    - **P1**: basal boundary of the ventricular zone (VZ)  
    - **P2**: apical surface (lumen)  
  - VZ thickness = distance P1–P2.  

- **Deformation Angle (β)**
  - Defined as angle between ellipse major axis and tissue radial axis through centroid.
  - Interpretation:  
    - β ≈ 0° → radial alignment (forces tangential)  
    - β ≈ 90° → tangential alignment (forces radial)  
  - Only droplets with eccentricity ≥ 0.35 included.

- **Movement Direction (θ)**
  - Centroid positions tracked across consecutive frames.
  - Displacement vectors = difference between centroids at T and T–4.
  - Local radial axis defined by P1–P2 at T–4.
  - θ = angle between displacement vector and radial axis (dot product), constrained to 0°–90°:
    - 0° → radial movement  
    - 90° → tangential movement  
  - Angles only calculated if:
    1. At least 4 frames of trajectory history available  
    2. Displacement ≥ half the droplet diameter  

## Outputs
- Droplet radius (per time point)  
- Droplet position within the VZ  
- Deformation angles β (only for eccentric droplets)  
- Movement angles θ (for sufficiently large displacements)  

In [ ]:
import sys, os, tqdm
import numpy as np
import pandas as pd
import napari
from PyQt5 import QtCore, QtGui, QtWidgets

from skimage.io import imread
from skimage.morphology import remove_small_holes, remove_small_objects,\
                            binary_opening, disk
from skimage import filters
from skimage import measure

from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
# === Define pixel resolution of the imported image
pxl_size = 0.4874 # in um/pixel

### Step1: Load tif file

In [ ]:
app = QtCore.QCoreApplication.instance()
if app is None:
    app = QtWidgets.QApplication(sys.argv)
fname = QtWidgets.QFileDialog.getOpenFileName(None, "Select a file...", '.', filter="All files (*)")[0]
print('Chosen file:', fname)

In [ ]:
# Replace with the actual image pathway (copy from the printed fname from above
fname = '/////image-name.tif' 

### Step2: Read image & identify the channel for processing (i.e., droplet channel)

In [ ]:
movie = imread(fname)
print('Movie shape:', movie.shape) # movie.shape(time, Z slice number, channel number, hieght, width)
channels = [movie[:,:,i,:,:] for i in range(movie.shape[2])]

### Step3: Create a mask for droplet and make mask as additional channel¶

In [ ]:
filters.threshold_otsu(channels[-1]) # droplet channel

In [ ]:
mask = channels[-1] # droplet channel
mask = channels[-1] > (filters.threshold_otsu(channels[-1])-7500) # dmodify the threshold value based on needs

for t in range(mask.shape[0]):
    for z in range(mask.shape[1]):
        mask[t,z] = remove_small_holes(mask[t,z], 100)
        mask[t,z] = remove_small_objects(mask[t,z], 100)
        mask[t,z] = binary_opening(mask[t,z], disk(3))
channels.append(mask)

print('Number of channels (including mask):', len(channels))
print('Image size XY:', channels[-1].shape[-1])
print('Image size Z:', channels[-1].shape[1])
print('Timepoints:', channels[-1].shape[0])

### Step4: Visualize image in Napari, add 2 reference points

In [ ]:
viewer = napari.Viewer()
i=0
colors = ['green', 'magenta', 'red', 'gray', 'cyan']
for channel in channels[:-1]: 
    viewer.add_image(channel, name='channel%02d'%i, blending='additive', colormap=colors[i], scale=(10,1,1))
    i+=1
viewer.add_labels(channels[-1], name='channel%02d'%i, blending='additive', scale=(10,1,1))
napari.run()

### Step5: Filter frames where only 2 points are marked, preparing for downstream analysis

In [ ]:
points = viewer.layers['Points'].data.astype(np.uint16) # time, slice, X, Y

ts = list(set(points[:,0]))
ts.sort()
points_filt = []
for t in ts:
    ps = points[points[:,0]==t,:]
    if ps.shape[0]==2:
        points_filt.append(ps[0])
        points_filt.append(ps[1])
points_filt = np.array(points_filt)


### Step6: Visulize the image, annotated reference points and the connecting line

In [ ]:
ts = [int(i) for i in points_filt[::2,0]]
zs = [int(i) for i in points_filt[::2,1]]

fig, ax = plt.subplots(nrows=len(ts), ncols=len(channels), figsize=(3*len(channels), 3*len(ts)))

plt.subplots_adjust(hspace=0.4, wspace=0.4)

plt.tight_layout()

if len(ts) == 1:
    ax = np.array([ax])
    
for j in range(len(ts)):
    points_t = points_filt[points_filt[:,0]==ts[j]]
    for i in range(len(channels)):
        ax[j,i].imshow(channels[i][ts[j],zs[j]])
        ax[j,i].plot(points_t[:,3],points_t[:,2],'-ow')
        ax[j,i].set_title("t %d"%(ts[j]))
        
plt.show()

### Step7: Analyze droplet geometry and visualise the ellipse-fitting and annotation outcome

In [ ]:
def process_plot_and_save_droplet_data(ts, zs, points_filt, channels, pxl_size, fname):
    dfs = pd.DataFrame({})

    for j in tqdm.tqdm(range(len(ts))):
        print('----- index: ', j, ' timepoint: ', ts[j])  
        labels = measure.label(channels[-1][ts[j]][zs[j]])  
        
        props = pd.DataFrame(measure.regionprops_table(labels, properties=(
            'label', 'bbox', 'centroid', 'axis_major_length', 'axis_minor_length', 'orientation')))
        
        points_t = points_filt[points_filt[:, 0] == ts[j]]
        p1 = points_t[0, 2:]  
        p2 = points_t[1, 2:] 
        p1_um = points_t[0, 2:] * pxl_size  
        p2_um = points_t[1, 2:] * pxl_size  
        
        # Calculate droplet centroid pos 
        centroid = np.array([props['centroid-0'].values, props['centroid-1'].values]).T
        centroid_um = centroid[idx_closer] * pxl_size
        
        if centroid.size > 0:
            # Calculate distances to p1 and find the closest droplet
            d = [np.linalg.norm(i - p1) for i in centroid]
            idx_closer = np.where(d == np.min(d))[0][0]
            
            # Calculate axis lengths in micrometers
            axis_minor_length = props.loc[idx_closer, 'axis_minor_length']
            axis_major_length = props.loc[idx_closer, 'axis_major_length']
            axis_minor_length_um = axis_minor_length * pxl_size
            axis_major_length_um = axis_major_length * pxl_size

            # Calculate droplet radius
            droplet_radius_um = (axis_minor_length_um + axis_major_length_um) / 4

            # Calculate eccentricity 
            eccentricity = np.sqrt(1 - (axis_minor_length / axis_major_length) ** 2)

            # Calculate distances for 'neuroepithelial thickness' and 'centroid to apical distance'
            neuroepi_thickness = np.linalg.norm(p2_um - p1_um)
            centroid_to_apical_distance = np.linalg.norm(centroid_um - p2_um)
        
            # Calculate reference vector from p1 to the centroid (i.e., radial axis passing through centroid)
            reference_vector = centroid[idx_closer] - p1
            reference_vector_norm = np.linalg.norm(reference_vector)
            if reference_vector_norm > 0:
                reference_vector = reference_vector / reference_vector_norm 

            # Calculate the vactor representing droplet's major axis
            droplet_orientation_rad = props.loc[idx_closer, 'orientation']
            droplet_major_axis_vector = np.array([
                np.cos(droplet_orientation_rad),
                np.sin(droplet_orientation_rad)
            ])

            # Calculate the angle between the reference vector and the droplet's major axis vector
            dot_product = np.dot(reference_vector, droplet_major_axis_vector)
            deformation_angle_rad = np.arccos(np.clip(dot_product, -1.0, 1.0))  # Angle in radians

            # Convert the angle from radians to degrees, and transform/constrained to 90 deg 
            deformation_angle_deg = np.degrees(deformation_angle_rad)  
            if deformation_angle_deg > 90:
                deformation_angle_deg = 180 - deformation_angle_deg

            # Filter angle deviation, excluding droplet without clear deformation based on eccentricity
            if eccentricity >= 0.35:
                deformation_angle_deviation_deg_filtered = deformation_angle_deg
            else:
                deformation_angle_deviation_deg_filtered = 'N/A'

            # Generate the dataframe for this timepoint
            df_selected = pd.DataFrame({
                'tp': ts[j],
                'z': zs[j],
                'p1_x': p1[0],
                'p1_y': p1[1],
                'p2_x': p2[0],
                'p2_y': p2[1],
                'centroid_x': centroid[idx_closer, 0],
                'centroid_y': centroid[idx_closer, 1],
                'axis_major_length': axis_major_length,
                'axis_minor_length': axis_minor_length,
                'axis_major_length_um': axis_major_length_um,
                'axis_minor_length_um': axis_minor_length_um,
                'eccentricity': eccentricity,
                'neuroepi_thickness_um': neuroepi_thickness,
                'centroid_to_apical_distance_um': centroid_to_apical_distance,
                'droplet_radius_um': droplet_radius_um,
                'deformation_angle_deviation_deg': deformation_angle_deg,  
                'deformation_angle_deviation_deg_filtered': deformation_angle_deviation_deg_filtered  n
            }, index=[0])

            dfs = pd.concat([dfs, df_selected], axis=0, ignore_index=True)

            # Plotting for the current timepoint
            fig, ax = plt.subplots()
            
            # Extract the centroid coordinates for the closest droplet
            centroid_x = centroid[idx_closer, 0]
            centroid_y = centroid[idx_closer, 1]

            # Plot the droplet's major and minor axes as an ellipse with the correct orientation
            ellipse = Ellipse((centroid_x, centroid_y), 
                              width=axis_major_length, 
                              height=axis_minor_length,
                              angle=np.degrees(droplet_orientation_rad),  # Correct orientation
                              edgecolor='black', 
                              facecolor='none', 
                              lw=2)
            ax.add_patch(ellipse)
            
            # Calculate the endpoints of the major and minor axes
            major_dx = axis_major_length / 2
            major_dy = axis_minor_length / 2

            # Calculate major axis endpoints
            major_axis_x = [centroid_x - major_dx * np.cos(droplet_orientation_rad), 
                             centroid_x + major_dx * np.cos(droplet_orientation_rad)]
            major_axis_y = [centroid_y - major_dx * np.sin(droplet_orientation_rad), 
                             centroid_y + major_dx * np.sin(droplet_orientation_rad)]

            # Plot the major axis (green line)
            ax.plot(major_axis_x, major_axis_y, 'g-', lw=2)

            # Calculate minor axis endpoints
            minor_axis_x = [centroid_x + major_dy * np.sin(droplet_orientation_rad), 
                             centroid_x - major_dy * np.sin(droplet_orientation_rad)]
            minor_axis_y = [centroid_y - major_dy * np.cos(droplet_orientation_rad), 
                             centroid_y + major_dy * np.cos(droplet_orientation_rad)]

            # Plot the droplet minor axis (**red line**)
            ax.plot(minor_axis_x, minor_axis_y, 'r-', lw=2)
            
            # Plot the droplet major axis (**green line**)
            ax.plot(major_axis_x, major_axis_y, 'g-', lw=2)
            
            # Plot the line linking p1 and p2 (**orange line**)
            ax.plot([p1[0], p2[0]], [p1[1], p2[1]], 'orange', lw=2)

            # Plot the reference vector from p1 to centroid (i.e., radial axis, **blue line**)
            ax.plot([p1[0], centroid_x], [p1[1], centroid_y], 'b-', lw=2)

            ax.set_aspect('equal')
            ax.set_title(f"Timepoint {ts[j]}, Z-slice {zs[j]}")
            ax.set_xlim(0, channels[-1][ts[j]][zs[j]].shape[1]) 
            ax.set_ylim(0, channels[-1][ts[j]][zs[j]].shape[0])  


            textstr = (f"tp: {ts[j]}, z: {zs[j]}\n"
                       f"p1_x: {p1[0]:.2f}, p1_y: {p1[1]:.2f}\n"
                       f"p2_x: {p2[0]:.2f}, p2_y: {p2[1]:.2f}\n"
                       f"Eccentricity: {eccentricity:.3f}\n"
                       f"Aspect Ratio: {aspect_ratio:.3f}\n"
                       f"Deformation Angle Deviation: {angle_deg:.2f} degrees\n"
                       f"Filtered Deformation Angle Deviation: {deformation_angle_deviation_deg_filtered}")
            ax.text(0.05, -0.3, textstr, transform=ax.transAxes, fontsize=10,
                    verticalalignment='top', bbox=dict(facecolor='white', alpha=0.5))

            plt.show()
        else:
            print(f"No droplets found at timepoint {ts[j]} and z-slice {zs[j]}")
            continue  # Skip to the next iteration if no droplets are found
    
    # Print the final DataFrame
    print("\nFinal DataFrame (dfs):")
    print(dfs)

    # Save the results to an Excel file
    output_file = os.path.join(os.path.split(fname)[0], 'output_' + os.path.splitext(os.path.split(fname)[-1])[0] + '.xlsx')
    dfs.to_excel(output_file, index=False)
    print(f"\nResults saved to {output_file}")
    
    return dfs

# output includes droplet radius, position and deformation angles β 

In [ ]:
# call the defined function 
dfs = process_plot_and_save_droplet_data(ts, zs, points_filt, channels, pxl_size, fname)

### Step8: calculate droplet movement angle

In [ ]:
def plot_and_save_angle_deviated_from_radial_axis_T4(dfs, fname, droplet_diameter, compare_interval=4):
    movement_angle_deviations_all = [] # angle between the radial axis and the vector connecting droplet consecutive centroids
    movement_angle_deviations_filtered = []
    timepoints = dfs['tp'].values[compare_interval:]  
    centroids = dfs[['centroid_x', 'centroid_y']].values
    centroids_um = centroids * pxl_size
    p1s = dfs[['p1_x', 'p1_y']].values    

    for i in range(compare_interval, len(centroids)):
        droplet_moving_distance = np.linalg.norm(centroids_um[i] - centroids_um[i-compare_interval])

        # Reference vector at T-4
        reference_vector_previous_t = centroids_um[i-compare_interval] - p1s[i-compare_interval]
        reference_vector_previous_t = reference_vector_previous_t / np.linalg.norm(reference_vector_previous_t)

        # Moving axis at T (comparing with T-4)
        moving_axis = centroids_um[i] - centroids_um[i-compare_interval]
        moving_axis = moving_axis / np.linalg.norm(moving_axis)

        # Movement angle deviation calculation
        dot_product = np.dot(reference_vector_previous_t, moving_axis)
        movement_angle_deviation = np.arccos(np.clip(dot_product, -1.0, 1.0)) * 180 / np.pi  # Convert to degrees

        # Ensure the angle is within the range of 0-90 degrees
        if movement_angle_deviation > 90:
            movement_angle_deviation = 180 - movement_angle_deviation

        movement_angle_deviations_all.append(movement_angle_deviation)

        # Include only droplet_moving_distance > 0.5 * droplet_diameter
        if droplet_moving_distance > 0.5 * droplet_diameter:
            movement_angle_deviations_filtered.append(movement_angle_deviation)
        else:
            print(f"At t[{dfs['tp'].values[i]}], droplet moved less than half its diameter")
            movement_angle_deviations_filtered.append('N/A')  # Use 'N/A' instead of None

    # The first entries for angle deviation need to be filled with 'N/A' for time points where comparison is not possible
    movement_angle_deviations_all = ['N/A'] * compare_interval + movement_angle_deviations_all
    movement_angle_deviations_filtered = ['N/A'] * compare_interval + movement_angle_deviations_filtered

    # Create DataFrames containing the relevant columns
    movement_angle_df = pd.DataFrame({
        'tp': dfs['tp'].values,
        'angle_deviation_from_radial_axis_all': movement_angle_deviations_all,
        'angle_deviation_from_radial_axis_filtered': movement_angle_deviations_filtered
    })

    # Create the output file name based on the original file name
    base_name = os.path.splitext(os.path.basename(fname))[0]
    output_file_combined = os.path.join(os.path.split(fname)[0], f"{base_name}_movement_angle_deviation_T_vs_T4_combined.xlsx")
    movement_angle_df.to_excel(output_file_combined, index=False)
    print(f"\nCombined angle deviation (T vs T-4) saved to {output_file_combined}")

    plt.figure(figsize=(10, 6))
    plt.plot(dfs['tp'].values, movement_angle_deviations_all, 'o-', color='blue', markersize=8, label='Angle Deviation (All)')
    plt.plot(dfs['tp'].values, movement_angle_deviations_filtered, 'o-', color='red', markersize=8, label='Angle Deviation (Filtered)')
    plt.xlabel('Time (t)')
    plt.ylabel('Movement Angle Deviation from Radial Axis (degrees)')
    plt.title('Movement Angle Deviation from Radial Axis Over Time (Comparing T and T-4)')
    plt.legend()
    plt.show()


In [ ]:
# call function
droplet_diameter = np.average(dfs['axis_major_length_um'].values + dfs['axis_major_length_um'].values)
plot_and_save_angle_deviated_from_radial_axis_T4(dfs, fname, droplet_diameter, compare_interval=4)
